In [39]:
### DEFINING EXTRACTION UNIVERSE DATA FROM GENERAL MS EXCEL SOURCE
def get_market_membership_from_excel(path_msci):
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd
    ### Declaring local constants & variables: 
    tab_monthly = 'monthly_data'    
    arr_markets_needed = ['DM', 'FM', 'EM']   
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM'}
    no_slice = slice(None)
    ### Extracting universe data:
    df_universe = pd.read_excel(io = path_msci, sheet_name = tab_monthly, skiprows = [0, 2], header = 0, parse_dates = True, 
                                na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'])
    df_universe = df_universe.loc[no_slice, ['dates', 'region', 'ctry']]
    df_universe.columns = ['Date', 'Market', 'Country']
    df_universe.set_index(['Date', 'Country'], inplace = True)
    ser_universe = df_universe.squeeze()
    ser_universe.sort_index(level = [0, 1], inplace = True)
    ser_universe.replace(dict_markets, inplace = True)
    ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
    
    return ser_market_membership

In [40]:
### DEFINING EXTRACTION RATING DATA FROM GENERAL MS EXCEL SOURCE
def get_ratings_from_excel(path_xsdm, path_scale, path_msci):
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd    
    ### Declaring local constants & variables:  
    arr_currency = ['LC', 'FC'] ### For correct sheet selection in XSDM file
    arr_to_skip = list(range(5)) ### For rows skipping in XSDM file
    arr_to_subtract = ['\(P\)', 'u', ' \*.?$'] ### For ratings clearing 
    str_to_subtract_excess = '|'.join(arr_to_subtract)
    str_to_highlight_mark = '(\*.)'  ### For future reviews markers extracting
    dict_map_to_replace = {'DD': 'D', 'NR': np.NaN, 'RD': 'D', 'SD': 'D', 'WD': np.NaN, 'WR': np.NaN, 'nan': np.NaN} ### For ratings harmonizing with scaling table 
    dict_table = {} ### For sheets accumulation
    ### Collecting sheets:
    with pd.ExcelFile(path_xsdm) as file_excel:
        for str_sheet_name in file_excel.sheet_names:
            arr_sheet_code = str_sheet_name.split(' ')[0].split('_')
            if (len(arr_sheet_code) > 2):
                if (arr_sheet_code[1] in arr_currency):
                    ser_table = pd.read_excel(file_excel, str_sheet_name, squeeze = True, skiprows = arr_to_skip, index_col = 0, parse_dates = True)
                    dict_table[str_sheet_name.split(' ')[0]] = ser_table.stack(dropna = False).sort_index()
    ### Aggregating tables:
    ser_common = pd.concat(dict_table)
    ser_common.index.names = ['Code', 'Date', 'Country']
    ser_common = ser_common.astype(str)
    ### Splitting 'Rating Code' for future analysis:
    df_common = ser_common.to_frame().reset_index().set_index('Code')
    df_common.index = pd.MultiIndex.from_tuples(df_common.index.str.split('_').to_list())
    df_common.index.names = ['Agency', 'Currency', 'Type']
    ### Rating dataFrame polishing:
    df_rating = df_common.set_index(['Date', 'Country'], drop = True, append = True).sort_index()        
    df_rating.columns = ['Rating']
    ### Future reviews markers extracting:
#    df_rating['Initial'] = df_rating['Rating']
    df_rating['Mark'] = df_rating['Rating'].str.extract(pat = str_to_highlight_mark, expand = False).str.slice(-1)    
    ### Suffixes and prefixes handling:    
    df_rating['Rating'] = df_rating['Rating'].str.replace(pat = str_to_subtract_excess, repl = '', regex = True)
    ### Ratings harmonizing with scaling table:
    for map_key, map_value in dict_map_to_replace.items():
        df_rating['Rating'] = df_rating['Rating'].replace(map_key, map_value)
    ### Reading scaling table:
    df_scale = pd.read_excel(path_scale, index_col = 'Rank')
    ### Scaling dataFrame polishing:    
    df_scale.rename_axis('Agency', axis = 1, inplace = True)
    df_scale = df_scale.stack(dropna = False).to_frame().swaplevel().reset_index(1)
    df_scale.rename(columns = {0 : 'Rating'}, inplace = True)
    df_scale.set_index('Rating', append = True, inplace = True)
    ### Adding rank column to result dataframe:
    df_rating = df_rating.join(df_scale, on = ['Agency', 'Rating'], how = 'left')    
#    df_result.drop(['Initial', 'Rating'], axis = 1, inplace = True)
    df_rating.drop(['Rating'], axis = 1, inplace = True)  
    ### Receiving MSCI membership data:    
    ser_market_membership = get_market_membership_from_excel(path_msci)
    ### Adding membership column to result dataframe:    
    df_result = df_rating.join(ser_market_membership, on = ['Date', 'Country'], how = 'left')
    df_result = df_result.set_index('Market', drop = True, append = True)    
    ### Reordering indexes and value columns:
    df_result = df_result[['Rank', 'Mark']]
    df_result = df_result.reorder_levels(['Agency', 'Currency', 'Type', 'Date', 'Market', 'Country'])
    return df_result

In [41]:
### TESTER
### Importing standard modules and date-special modules:
import numpy as np
import pandas as pd
### Declaring global constants & variables: 
path_msci = 'Data_Files/Source_Files/Sample_Membership.xlsx' ### Path for membership source
path_xsdm = 'Data_Files/Source_Files/XSDM.xlsx' ### Path to rating data file
path_scale = 'Data_Files/Source_Files/Ratings_Scale.xlsx' ### Path to rating comparision file 
All = slice(None)
### Receiving aggregated ratings table
df_rating_full = get_ratings_from_excel(path_xsdm, path_scale, path_msci)
print(df_rating_full.tail())

                                                  Rank Mark
Agency Currency Type   Date       Market Country           
SP     LC       ISSUER 2019-08-30 NaN    UG       -5.0  NaN
                                  DM     US        9.0  NaN
                                  FM     VN       -2.0  NaN
                                  EM     ZA       -1.0  NaN
                                  NaN    ZM       -7.0  NaN


In [116]:
### TESTER
df_rating = df_rating_full.loc[(All, All, All, All, ['DM', 'FM', 'EM'], All), :]
df_rating = df_rating.unstack(level = 'Currency')
df_rating.sort_index(level = ['Agency', 'Type', 'Date', 'Country'], inplace = True)
df_rating.columns = ['_'.join(iter_tup_col).rstrip('_') for iter_tup_col in df_rating.columns.values]
print(df_rating.tail())

                                       Rank_FC  Rank_LC Mark_FC Mark_LC
Agency Type Date       Market Country                                  
FITCH  DEBT 1994-01-31 DM     AT           NaN      NaN     NaN     NaN
                              AU           NaN      NaN     NaN     NaN
                              BE           NaN      NaN     NaN     NaN
                              CA           NaN      NaN     NaN     NaN
                              CH           NaN      NaN     NaN     NaN
                                         Rank_FC  Rank_LC Mark_FC Mark_LC
Agency Type   Date       Market Country                                  
SP     ISSUER 2019-08-30 EM     TR          -4.0     -3.0     NaN     NaN
                                TW           7.0      7.0     NaN     NaN
                         DM     US           9.0      9.0     NaN     NaN
                         FM     VN          -2.0     -2.0     NaN     NaN
                         EM     ZA          -2.0    

In [120]:
### TESTER
df_rating['Rank_FC'].fillna(df_rating['Rank_LC'], inplace = True)
df_rating.loc[('SP', 'ISSUER', '2001-09-28', All, 'IE')]

,,,,,Rank_FC,Rank_LC,Mark_FC,Mark_LC
Agency,Type,Date,Market,Country,,,,
SP,ISSUER,2001-09-28,DM,IE,10.0,10.0,NaN,NaN


In [96]:
### TESTER

In [74]:
### TESTER
df_sp = df_rating_full.loc[('SP', 'FC')].merge(df_rating_full.loc[('SP', 'LC')], how = 'outer', left_index = True, right_index = True, suffixes = ('_FC', '_LC'))
df_sp.reset_index(level = 'Type', drop = True, inplace = True)
df_sp = df_sp.loc[(All, ['DM', 'FM', 'EM'], All), :]
df_sp.fillna(0, inplace = True)
df_sp['Rank_Delta'] = (df_sp['Rank_FC'] - df_sp['Rank_LC'])
print('SP ISSUER ISON UNIVERSE MEMBERS SIZE:', df_sp['Rank_FC'].size)
print('SP ISSUER ISON UNIVERSE MEMBERS FC NOT NAN COUNT:', df_sp['Rank_FC'][df_sp['Rank_FC'] != 0].count())
print('SP ISSUER ISON UNIVERSE MEMBERS LC NOT NAN COUNT:', df_sp['Rank_LC'][df_sp['Rank_LC'] != 0].count())
print('SP ISSUER ISON UNIVERSE MEMBERS FC vs LC COUNT DELTA:', (df_sp['Rank_FC'][df_sp['Rank_FC'] != 0].count() - df_sp['Rank_LC'][df_sp['Rank_LC'] != 0].count()))
print('SP ISSUER ISON UNIVERSE MEMBERS FC VALUED & LC NAN COUNT:', (df_sp['Rank_FC'][(df_sp['Rank_FC'] != 0) & (df_sp['Rank_LC'] == 0)].count()))
print('SP ISSUER ISON UNIVERSE MEMBERS FC NAN & LC VALUED COUNT:', (df_sp['Rank_LC'][(df_sp['Rank_FC'] == 0) & (df_sp['Rank_LC'] != 0)].count()))
ser_sp_rank = df_sp['Rank_Delta'][(df_sp['Rank_FC'] != 0) & (df_sp['Rank_LC'] != 0)]
print('SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED COUNT:', ser_sp_rank.size)
print('SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED EQUAL:', ser_sp_rank[ser_sp_rank == 0].size)
print('SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL:', ser_sp_rank[ser_sp_rank.abs() > 0].size)
print('SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL FC ADVANTAGE:', ser_sp_rank[ser_sp_rank > 0].size)
print('SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL LC ADVANTAGE:', ser_sp_rank[ser_sp_rank < 0].size)
print('SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM:', ser_sp_rank.abs().sum())
print('SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY:', '\n', ser_sp_rank.abs().groupby('Market').sum().sort_index())
print('SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS:', '\n', ser_sp_rank.abs().groupby('Country').sum().sort_values(ascending = False).head())
#ser_sp_rank.to_excel('Data_Files/Test_Files/SP_Ranking.xlsx', merge_cells = False)

SP ISSUER ISON UNIVERSE MEMBERS SIZE: 16374
SP ISSUER ISON UNIVERSE MEMBERS FC NOT NAN COUNT: 15729
SP ISSUER ISON UNIVERSE MEMBERS LC NOT NAN COUNT: 15720
SP ISSUER ISON UNIVERSE MEMBERS FC vs LC COUNT DELTA: 9
SP ISSUER ISON UNIVERSE MEMBERS FC VALUED & LC NAN COUNT: 83
SP ISSUER ISON UNIVERSE MEMBERS FC NAN & LC VALUED COUNT: 74
SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED COUNT: 15646
SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED EQUAL: 10056
SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL: 5590
SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL FC ADVANTAGE: 0
SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL LC ADVANTAGE: 5590
SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM: 10014.0
SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY: 
 Market
DM    1402.0
EM    7469.0
FM    1143.0
Name: Rank_Delta, dtype: float64
SP ISSUER ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS: 
 Country
CO    619.0
MX    570.0
ZA    545.0
CL    522.0
JO    486.0


In [130]:
#df_sp[(df_sp['Rank_FC'] != 0) & (df_sp['Rank_LC'] == 0)].sort_index(level = ['Country', 'Date'])
#df_sp[(df_sp['Rank_FC'] == 0) & (df_sp['Rank_LC'] != 0)].sort_index(level = ['Country', 'Date'])
print(df_sp[(df_sp['Rank_FC'] == 0) & (df_sp['Rank_LC'] != 0)].index.get_level_values(2).unique())
print(df_sp[(df_sp['Rank_FC'] == 0) & (df_sp['Rank_LC'] != 0)].index.get_level_values(2).size)
print(df_sp[(df_sp['Rank_FC'] == 0) & (df_sp['Rank_LC'] != 0)].index.get_level_values(0).unique()[0])
print(df_sp[(df_sp['Rank_FC'] == 0) & (df_sp['Rank_LC'] != 0)].index.get_level_values(0).unique()[-1])
print(df_sp[(df_sp['Rank_FC'] == 0) & (df_sp['Rank_LC'] != 0)]['Mark_LC'].unique())
df_sp.loc[('2001-10-31', All, 'IE'), :]

Index(['IE'], dtype='object', name='Country')
74
1995-08-31 00:00:00
2001-09-28 00:00:00
[0]


,,,Rank_FC,Mark_FC,Rank_LC,Mark_LC,Rank_Delta
Date,Market,Country,,,,,
2001-10-31,DM,IE,10.0,0,10.0,0,0.0


In [44]:
### TESTER
df_fitch = df_rating_full.loc[('FITCH', 'FC')].merge(df_rating_full.loc[('FITCH', 'LC')], how = 'outer', left_index = True, right_index = True, suffixes = ('_FC', '_LC'))
df_fitch.reset_index(level = 'Type', drop = True, inplace = True)
df_fitch = df_fitch.loc[(All, ['DM', 'FM', 'EM'], All), :]
df_fitch.fillna(0, inplace = True)
df_fitch['Rank_Delta'] = (df_fitch['Rank_FC'] - df_fitch['Rank_LC'])
print('FITCH DEBT ISON UNIVERSE MEMBERS SIZE:', df_fitch['Rank_FC'].size)
print('FITCH DEBT ISON UNIVERSE MEMBERS FC NOT NAN COUNT:', df_fitch['Rank_FC'][df_fitch['Rank_FC'] != 0].count())
print('FITCH DEBT ISON UNIVERSE MEMBERS LC NOT NAN COUNT:', df_fitch['Rank_LC'][df_fitch['Rank_LC'] != 0].count())
print('FITCH DEBT ISON UNIVERSE MEMBERS FC vs LC COUNT DELTA:', 
      (df_fitch['Rank_FC'][df_fitch['Rank_FC'] != 0].count() - df_fitch['Rank_LC'][df_fitch['Rank_LC'] != 0].count()))
print('FITCH DEBT ISON UNIVERSE MEMBERS FC VALUED & LC NAN COUNT:', (df_fitch['Rank_FC'][(df_fitch['Rank_FC'] != 0) & (df_fitch['Rank_LC'] == 0)].count()))
print('FITCH DEBT ISON UNIVERSE MEMBERS FC NAN & LC VALUED COUNT:', (df_fitch['Rank_LC'][(df_fitch['Rank_FC'] == 0) & (df_fitch['Rank_LC'] != 0)].count()))
ser_fitch_rank = df_fitch['Rank_Delta'][(df_fitch['Rank_FC'] != 0) & (df_fitch['Rank_LC'] != 0)]
print('FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED COUNT:', ser_fitch_rank.size)
print('FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED EQUAL:', ser_fitch_rank[ser_fitch_rank == 0].size)
print('FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL:', ser_fitch_rank[ser_fitch_rank.abs() > 0].size)
print('FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL FC ADVANTAGE:', ser_fitch_rank[ser_fitch_rank > 0].size)
print('FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL LC ADVANTAGE:', ser_fitch_rank[ser_fitch_rank < 0].size)
print('FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM:', ser_fitch_rank.abs().sum())
print('FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY:', '\n', ser_fitch_rank.abs().groupby('Market').sum().sort_index())
print('FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS:', '\n', ser_fitch_rank.abs().groupby('Country').sum().sort_values(ascending = False).head())
#ser_fitch_rank.to_excel('Data_Files/Test_Files/SP_Ranking.xlsx', merge_cells = False)

FITCH DEBT ISON UNIVERSE MEMBERS SIZE: 16374
FITCH DEBT ISON UNIVERSE MEMBERS FC NOT NAN COUNT: 14305
FITCH DEBT ISON UNIVERSE MEMBERS LC NOT NAN COUNT: 13950
FITCH DEBT ISON UNIVERSE MEMBERS FC vs LC COUNT DELTA: 355
FITCH DEBT ISON UNIVERSE MEMBERS FC VALUED & LC NAN COUNT: 569
FITCH DEBT ISON UNIVERSE MEMBERS FC NAN & LC VALUED COUNT: 214
FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED COUNT: 13736
FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED EQUAL: 8434
FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL: 5302
FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL FC ADVANTAGE: 165
FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL LC ADVANTAGE: 5137
FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM: 7807.0
FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY: 
 Market
DM    1812.0
EM    4509.0
FM    1486.0
Name: Rank_Delta, dtype: float64
FITCH DEBT ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS: 
 Country
CO    391.0
HK    351.0
EG    340.0
TH    

In [117]:
print(df_fitch[(df_fitch['Rank_FC'] == 0) & (df_fitch['Rank_LC'] != 0)].index.get_level_values(2).unique())
print(df_fitch[(df_fitch['Rank_FC'] == 0) & (df_fitch['Rank_LC'] != 0)].index.get_level_values(2).size)
print(df_fitch[(df_fitch['Rank_FC'] == 0) & (df_fitch['Rank_LC'] != 0)].index.get_level_values(0).unique()[0])
print(df_fitch[(df_fitch['Rank_FC'] == 0) & (df_fitch['Rank_LC'] != 0)].index.get_level_values(0).unique()[-1])
print(df_fitch[(df_fitch['Rank_FC'] == 0) & (df_fitch['Rank_LC'] != 0)]['Mark_LC'].unique())

Index(['TW'], dtype='object', name='Country')
214
2001-11-30 00:00:00
2019-08-30 00:00:00
[0]


In [46]:
### TESTER
df_mdy_issuer = df_rating_full.loc[('MDY', 'FC', 'ISSUER')].merge(df_rating_full.loc[('MDY', 'LC', 'ISSUER')], 
                                                             how = 'outer', left_index = True, right_index = True, suffixes = ('_FC', '_LC'))
df_mdy_issuer = df_mdy_issuer.loc[(All, ['DM', 'FM', 'EM'], All), :]
df_mdy_issuer.fillna(0, inplace = True)
df_mdy_issuer['Rank_Delta'] = (df_mdy_issuer['Rank_FC'] - df_mdy_issuer['Rank_LC'])
print('MDY ISSUER ISON UNIVERSE MEMBERS SIZE:', df_mdy_issuer['Rank_FC'].size)
print('MDY ISSUER ISON UNIVERSE MEMBERS FC NOT NAN COUNT:', df_mdy_issuer['Rank_FC'][df_mdy_issuer['Rank_FC'] != 0].count())
print('MDY ISSUER ISON UNIVERSE MEMBERS LC NOT NAN COUNT:', df_mdy_issuer['Rank_LC'][df_mdy_issuer['Rank_LC'] != 0].count())
print('MDY ISSUER ISON UNIVERSE MEMBERS FC vs LC COUNT DELTA:', 
      (df_mdy_issuer['Rank_FC'][df_mdy_issuer['Rank_FC'] != 0].count() - df_mdy_issuer['Rank_LC'][df_mdy_issuer['Rank_LC'] != 0].count()))
print('MDY ISSUER ISON UNIVERSE MEMBERS FC VALUED & LC NAN COUNT:', 
      (df_mdy_issuer['Rank_FC'][(df_mdy_issuer['Rank_FC'] != 0) & (df_mdy_issuer['Rank_LC'] == 0)].count()))
print('MDY ISSUER ISON UNIVERSE MEMBERS FC NAN & LC VALUED COUNT:', 
      (df_mdy_issuer['Rank_LC'][(df_mdy_issuer['Rank_FC'] == 0) & (df_mdy_issuer['Rank_LC'] != 0)].count()))
ser_mdy_issuer_rank = df_mdy_issuer['Rank_Delta'][(df_mdy_issuer['Rank_FC'] != 0) & (df_mdy_issuer['Rank_LC'] != 0)]
print('MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED COUNT:', ser_mdy_issuer_rank.size)
print('MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED EQUAL:', ser_mdy_issuer_rank[ser_mdy_issuer_rank == 0].size)
print('MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL:', ser_mdy_issuer_rank[ser_mdy_issuer_rank.abs() > 0].size)
print('MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL FC ADVANTAGE:', ser_mdy_issuer_rank[ser_mdy_issuer_rank > 0].size)
print('MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL LC ADVANTAGE:', ser_mdy_issuer_rank[ser_mdy_issuer_rank < 0].size)
print('MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM:', ser_mdy_issuer_rank.abs().sum())
print('MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY:', '\n', ser_mdy_issuer_rank.abs().groupby('Market').sum().sort_index())
print('MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS:', '\n', ser_mdy_issuer_rank.abs().groupby('Country').sum().sort_values(ascending = False).head())
ser_mdy_issuer_rank.to_excel('Data_Files/Test_Files/MDY_ISSUER_Ranking.xlsx', merge_cells = False)

MDY ISSUER ISON UNIVERSE MEMBERS SIZE: 16374
MDY ISSUER ISON UNIVERSE MEMBERS FC NOT NAN COUNT: 14692
MDY ISSUER ISON UNIVERSE MEMBERS LC NOT NAN COUNT: 14182
MDY ISSUER ISON UNIVERSE MEMBERS FC vs LC COUNT DELTA: 510
MDY ISSUER ISON UNIVERSE MEMBERS FC VALUED & LC NAN COUNT: 621
MDY ISSUER ISON UNIVERSE MEMBERS FC NAN & LC VALUED COUNT: 111
MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED COUNT: 14071
MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED EQUAL: 12533
MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL: 1538
MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL FC ADVANTAGE: 263
MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL LC ADVANTAGE: 1275
MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM: 3441.0
MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY: 
 Market
DM     903.0
EM    2021.0
FM     517.0
Name: Rank_Delta, dtype: float64
MDY ISSUER ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS: 
 Country
JP    339.0
CO    310.0
PE    295.0
IN   

In [118]:
print(df_mdy_issuer[(df_mdy_issuer['Rank_FC'] == 0) & (df_mdy_issuer['Rank_LC'] != 0)])
print(df_mdy_issuer[(df_mdy_issuer['Rank_FC'] == 0) & (df_mdy_issuer['Rank_LC'] != 0)]['Mark_LC'].unique())

                           Rank_FC Mark_FC  Rank_LC Mark_LC  Rank_Delta
Date       Market Country                                              
1994-01-31 DM     NZ           0.0       0     10.0       0       -10.0
1994-02-28 DM     NZ           0.0       0     10.0       0       -10.0
1998-10-30 DM     HK           0.0       0      6.0       0        -6.0
1998-11-30 DM     HK           0.0       0      6.0       0        -6.0
1998-12-31 DM     HK           0.0       0      6.0       0        -6.0
1999-01-29 DM     HK           0.0       0      6.0       0        -6.0
1999-02-26 DM     HK           0.0       0      6.0       0        -6.0
1999-03-31 DM     HK           0.0       0      6.0       0        -6.0
1999-04-30 DM     HK           0.0       0      6.0       0        -6.0
1999-05-31 DM     HK           0.0       0      6.0       0        -6.0
1999-06-30 DM     HK           0.0       0      6.0       0        -6.0
1999-07-30 DM     AU           0.0       0     10.0       0     

In [48]:
### TESTER
df_mdy_debt = df_rating_full.loc[('MDY', 'FC', 'DEBT')].merge(df_rating_full.loc[('MDY', 'LC', 'DEBT')], 
                                                             how = 'outer', left_index = True, right_index = True, suffixes = ('_FC', '_LC'))
df_mdy_debt = df_mdy_debt.loc[(All, ['DM', 'FM', 'EM'], All), :]
df_mdy_debt.fillna(0, inplace = True)
df_mdy_debt['Rank_Delta'] = (df_mdy_debt['Rank_FC'] - df_mdy_debt['Rank_LC'])
print('MDY DEBT ISON UNIVERSE MEMBERS SIZE:', df_mdy_debt['Rank_FC'].size)
print('MDY DEBT ISON UNIVERSE MEMBERS FC NOT NAN COUNT:', df_mdy_debt['Rank_FC'][df_mdy_debt['Rank_FC'] != 0].count())
print('MDY DEBT ISON UNIVERSE MEMBERS LC NOT NAN COUNT:', df_mdy_debt['Rank_LC'][df_mdy_debt['Rank_LC'] != 0].count())
print('MDY DEBT ISON UNIVERSE MEMBERS FC vs LC COUNT DELTA:', 
      (df_mdy_debt['Rank_FC'][df_mdy_debt['Rank_FC'] != 0].count() - df_mdy_debt['Rank_LC'][df_mdy_debt['Rank_LC'] != 0].count()))
print('MDY DEBT ISON UNIVERSE MEMBERS FC VALUED & LC NAN COUNT:', 
      (df_mdy_debt['Rank_FC'][(df_mdy_debt['Rank_FC'] != 0) & (df_mdy_debt['Rank_LC'] == 0)].count()))
print('MDY DEBT ISON UNIVERSE MEMBERS FC NAN & LC VALUED COUNT:', 
      (df_mdy_debt['Rank_LC'][(df_mdy_debt['Rank_FC'] == 0) & (df_mdy_debt['Rank_LC'] != 0)].count()))
ser_mdy_debt_rank = df_mdy_debt['Rank_Delta'][(df_mdy_debt['Rank_FC'] != 0) & (df_mdy_debt['Rank_LC'] != 0)]
print('MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED COUNT:', ser_mdy_debt_rank.size)
print('MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED EQUAL:', ser_mdy_debt_rank[ser_mdy_debt_rank == 0].size)
print('MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL:', ser_mdy_debt_rank[ser_mdy_debt_rank.abs() > 0].size)
print('MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL FC ADVANTAGE:', ser_mdy_debt_rank[ser_mdy_debt_rank > 0].size)
print('MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL LC ADVANTAGE:', ser_mdy_debt_rank[ser_mdy_debt_rank < 0].size)
print('MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM:', ser_mdy_debt_rank.abs().sum())
print('MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY:', '\n', ser_mdy_debt_rank.abs().groupby('Market').sum().sort_index())
print('MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS:', '\n', ser_mdy_debt_rank.abs().groupby('Country').sum().sort_values(ascending = False).head())
#ser_mdy_debt_rank.to_excel('Data_Files/Test_Files/MDY_DEBT_Ranking.xlsx', merge_cells = False)

MDY DEBT ISON UNIVERSE MEMBERS SIZE: 16374
MDY DEBT ISON UNIVERSE MEMBERS FC NOT NAN COUNT: 12430
MDY DEBT ISON UNIVERSE MEMBERS LC NOT NAN COUNT: 12573
MDY DEBT ISON UNIVERSE MEMBERS FC vs LC COUNT DELTA: -143
MDY DEBT ISON UNIVERSE MEMBERS FC VALUED & LC NAN COUNT: 1662
MDY DEBT ISON UNIVERSE MEMBERS FC NAN & LC VALUED COUNT: 1805
MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED COUNT: 10768
MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED EQUAL: 9372
MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL: 1396
MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL FC ADVANTAGE: 56
MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL LC ADVANTAGE: 1340
MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM: 2743.0
MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY: 
 Market
DM     620.0
EM    1925.0
FM     198.0
Name: Rank_Delta, dtype: float64
MDY DEBT ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS: 
 Country
CO    406.0
EG    258.0
ZA    244.0
CL    233.0
HR    161.0
Name: Ra

In [114]:
print(df_mdy_debt[(df_mdy_debt['Rank_FC'] == 0) & (df_mdy_debt['Rank_LC'] != 0)].index.get_level_values(2).unique())
print(df_mdy_debt[(df_mdy_debt['Rank_FC'] == 0) & (df_mdy_debt['Rank_LC'] != 0)]['Mark_LC'].unique())

Index(['JP', 'US', 'SG', 'CH', 'EG', 'IN', 'TW', 'MA', 'MU', 'KZ'], dtype='object', name='Country')
[0 '-' '+']


In [50]:
### TESTER
df_mdy_fc = df_rating_full.loc[('MDY', 'FC', 'DEBT')].merge(df_rating_full.loc[('MDY', 'FC', 'ISSUER')], 
                                                             how = 'outer', left_index = True, right_index = True, suffixes = ('_DEBT', '_ISSUER'))
df_mdy_fc = df_mdy_fc.loc[(All, ['DM', 'FM', 'EM'], All), :]
df_mdy_fc.fillna(0, inplace = True)
df_mdy_fc['Rank_Delta'] = (df_mdy_fc['Rank_DEBT'] - df_mdy_fc['Rank_ISSUER'])
print('MDY FC ISON UNIVERSE MEMBERS SIZE:', df_mdy_fc['Rank_DEBT'].size)
print('MDY FC ISON UNIVERSE MEMBERS DEBT NOT NAN COUNT:', df_mdy_fc['Rank_DEBT'][df_mdy_fc['Rank_DEBT'] != 0].count())
print('MDY FC ISON UNIVERSE MEMBERS ISSUER NOT NAN COUNT:', df_mdy_fc['Rank_ISSUER'][df_mdy_fc['Rank_ISSUER'] != 0].count())
print('MDY FC ISON UNIVERSE MEMBERS DEBT vs ISSUER COUNT DELTA:', 
      (df_mdy_fc['Rank_DEBT'][df_mdy_fc['Rank_DEBT'] != 0].count() - df_mdy_fc['Rank_ISSUER'][df_mdy_fc['Rank_ISSUER'] != 0].count()))
print('MDY FC ISON UNIVERSE MEMBERS DEBT VALUED & ISSUER NAN COUNT:', 
      (df_mdy_fc['Rank_DEBT'][(df_mdy_fc['Rank_DEBT'] != 0) & (df_mdy_fc['Rank_ISSUER'] == 0)].count()))
print('MDY FC ISON UNIVERSE MEMBERS DEBT NAN & ISSUER VALUED COUNT:', 
      (df_mdy_fc['Rank_ISSUER'][(df_mdy_fc['Rank_DEBT'] == 0) & (df_mdy_fc['Rank_ISSUER'] != 0)].count()))
ser_mdy_fc_rank = df_mdy_fc['Rank_Delta'][(df_mdy_fc['Rank_DEBT'] != 0) & (df_mdy_fc['Rank_ISSUER'] != 0)]
print('MDY FC ISON UNIVERSE MEMBERS BOTH VALUED COUNT:', ser_mdy_fc_rank.size)
print('MDY FC ISON UNIVERSE MEMBERS BOTH VALUED EQUAL:', ser_mdy_fc_rank[ser_mdy_fc_rank == 0].size)
print('MDY FC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL:', ser_mdy_fc_rank[ser_mdy_fc_rank.abs() > 0].size)
print('MDY FC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DEBT ADVANTAGE:', ser_mdy_fc_rank[ser_mdy_fc_rank > 0].size)
print('MDY FC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL ISSUER ADVANTAGE:', ser_mdy_fc_rank[ser_mdy_fc_rank < 0].size)
print('MDY FC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM:', ser_mdy_fc_rank.abs().sum())
print('MDY FC ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY:', '\n', ser_mdy_fc_rank.abs().groupby('Market').sum().sort_index())
print('MDY FC ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS:', '\n', ser_mdy_fc_rank.abs().groupby('Country').sum().sort_values(ascending = False).head())
#ser_mdy_fc_rank.to_excel('Data_Files/Test_Files/MDY_FC_Ranking.xlsx', merge_cells = False)

MDY FC ISON UNIVERSE MEMBERS SIZE: 16374
MDY FC ISON UNIVERSE MEMBERS DEBT NOT NAN COUNT: 12430
MDY FC ISON UNIVERSE MEMBERS ISSUER NOT NAN COUNT: 14692
MDY FC ISON UNIVERSE MEMBERS DEBT vs ISSUER COUNT DELTA: -2262
MDY FC ISON UNIVERSE MEMBERS DEBT VALUED & ISSUER NAN COUNT: 1031
MDY FC ISON UNIVERSE MEMBERS DEBT NAN & ISSUER VALUED COUNT: 3293
MDY FC ISON UNIVERSE MEMBERS BOTH VALUED COUNT: 11399
MDY FC ISON UNIVERSE MEMBERS BOTH VALUED EQUAL: 11317
MDY FC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL: 82
MDY FC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DEBT ADVANTAGE: 1
MDY FC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL ISSUER ADVANTAGE: 81
MDY FC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM: 248.0
MDY FC ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY: 
 Market
DM      7.0
EM    122.0
FM    119.0
Name: Rank_Delta, dtype: float64
MDY FC ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS: 
 Country
AR    119.0
PE     92.0
HU     24.0
NZ      7.0
PL      2.0
Name: Rank_De

In [ ]:
ser_mdy_fc_rank[ser_mdy_fc_rank.abs() > 0]

In [51]:
### TESTER
df_mdy_lc = df_rating_full.loc[('MDY', 'LC', 'DEBT')].merge(df_rating_full.loc[('MDY', 'LC', 'ISSUER')], 
                                                             how = 'outer', left_index = True, right_index = True, suffixes = ('_DEBT', '_ISSUER'))
df_mdy_lc = df_mdy_lc.loc[(All, ['DM', 'FM', 'EM'], All), :]
df_mdy_lc.fillna(0, inplace = True)
df_mdy_lc['Rank_Delta'] = (df_mdy_lc['Rank_DEBT'] - df_mdy_lc['Rank_ISSUER'])
print('MDY LC ISON UNIVERSE MEMBERS SIZE:', df_mdy_lc['Rank_DEBT'].size)
print('MDY LC ISON UNIVERSE MEMBERS DEBT NOT NAN COUNT:', df_mdy_lc['Rank_DEBT'][df_mdy_lc['Rank_DEBT'] != 0].count())
print('MDY LC ISON UNIVERSE MEMBERS ISSUER NOT NAN COUNT:', df_mdy_lc['Rank_ISSUER'][df_mdy_lc['Rank_ISSUER'] != 0].count())
print('MDY LC ISON UNIVERSE MEMBERS DEBT vs ISSUER COUNT DELTA:', 
      (df_mdy_lc['Rank_DEBT'][df_mdy_lc['Rank_DEBT'] != 0].count() - df_mdy_lc['Rank_ISSUER'][df_mdy_lc['Rank_ISSUER'] != 0].count()))
print('MDY LC ISON UNIVERSE MEMBERS DEBT VALUED & ISSUER NAN COUNT:', 
      (df_mdy_lc['Rank_DEBT'][(df_mdy_lc['Rank_DEBT'] != 0) & (df_mdy_lc['Rank_ISSUER'] == 0)].count()))
print('MDY LC ISON UNIVERSE MEMBERS DEBT NAN & ISSUER VALUED COUNT:', 
      (df_mdy_lc['Rank_ISSUER'][(df_mdy_lc['Rank_DEBT'] == 0) & (df_mdy_lc['Rank_ISSUER'] != 0)].count()))
ser_mdy_lc_rank = df_mdy_lc['Rank_Delta'][(df_mdy_lc['Rank_DEBT'] != 0) & (df_mdy_lc['Rank_ISSUER'] != 0)]
print('MDY LC ISON UNIVERSE MEMBERS BOTH VALUED COUNT:', ser_mdy_lc_rank.size)
print('MDY LC ISON UNIVERSE MEMBERS BOTH VALUED EQUAL:', ser_mdy_lc_rank[ser_mdy_lc_rank == 0].size)
print('MDY LC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL:', ser_mdy_lc_rank[ser_mdy_lc_rank.abs() > 0].size)
print('MDY LC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DEBT ADVANTAGE:', ser_mdy_lc_rank[ser_mdy_lc_rank > 0].size)
print('MDY LC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL ISSUER ADVANTAGE:', ser_mdy_lc_rank[ser_mdy_lc_rank < 0].size)
print('MDY LC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM:', ser_mdy_lc_rank.abs().sum())
print('MDY LC ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY:', '\n', ser_mdy_lc_rank.abs().groupby('Market').sum().sort_index())
print('MDY LC ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS:', '\n', ser_mdy_lc_rank.abs().groupby('Country').sum().sort_values(ascending = False).head())
#ser_mdy_fc_rank.to_excel('Data_Files/Test_Files/MDY_LC_Ranking.xlsx', merge_cells = False)

MDY LC ISON UNIVERSE MEMBERS SIZE: 16374
MDY LC ISON UNIVERSE MEMBERS DEBT NOT NAN COUNT: 12573
MDY LC ISON UNIVERSE MEMBERS ISSUER NOT NAN COUNT: 14182
MDY LC ISON UNIVERSE MEMBERS DEBT vs ISSUER COUNT DELTA: -1609
MDY LC ISON UNIVERSE MEMBERS DEBT VALUED & ISSUER NAN COUNT: 1199
MDY LC ISON UNIVERSE MEMBERS DEBT NAN & ISSUER VALUED COUNT: 2808
MDY LC ISON UNIVERSE MEMBERS BOTH VALUED COUNT: 11374
MDY LC ISON UNIVERSE MEMBERS BOTH VALUED EQUAL: 11326
MDY LC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL: 48
MDY LC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DEBT ADVANTAGE: 36
MDY LC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL ISSUER ADVANTAGE: 12
MDY LC ISON UNIVERSE MEMBERS BOTH VALUED NOT EQUAL DIFFERENCES SUM: 84.0
MDY LC ISON UNIVERSE MEMBERS BOTH VALUED MARKET SUMMARY: 
 Market
DM    10.0
EM    74.0
FM     0.0
Name: Rank_Delta, dtype: float64
MDY LC ISON UNIVERSE MEMBERS BOTH VALUED DELTA LEADERS: 
 Country
TH    70.0
CA    10.0
RU     2.0
EG     2.0
ZA     0.0
Name: Rank_Delta, dty

In [103]:
### TESTER
gb_rating = df_rating_full.groupby(['Agency', 'Currency', 'Type'])
#gb_rating.groups.keys()
#print('Groups length:','\n', gb_rating.size())
#print('Groups not NaN count:','\n', gb_rating.count())
print('Groups nominal properties:','\n', gb_rating.agg(['size', 'count']).sort_index(level = ['Agency', 'Currency', 'Type']))

Groups nominal properties: 
                          Rank          Mark      
                         size  count   size count
Agency Currency Type                             
FITCH  FC       DEBT    39984  18984  39984   191
       LC       DEBT    39984  17932  39984   137
MDY    FC       DEBT    39984  18892  39984   391
                ISSUER  39984  19367  39984   319
       LC       DEBT    39984  15586  39984   267
                ISSUER  39984  17562  39984   266
SP     FC       ISSUER  39984  24171  39984   122
       LC       ISSUER  39984  21587  39984   265
